In [0]:
!pip install -q kss
%tensorflow_version 2.x

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import os
   
# PyDrive Authentication
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
folder_id = '1wV4k91wu5NVNuC-KHSBv2xxqpFSVd0gt'
   
def ListFolder(parent):
    filelist=[]
    file_list = drive.ListFile({'q': "'%s' in parents and trashed=false" % parent}).GetList()
    for f in file_list:
        if f['mimeType']=='application/vnd.google-apps.folder': # if folder
            filelist.append({"id":f['id'],"title":f['title'],"list":ListFolder(f['id'])})
        else:
            filelist.append({"title":f['title'],"id":f['id']})
    return filelist
   
   
file_lists = ListFolder(folder_id)

In [14]:
file_lists_from_drive = ListFolder(folder_id)
   
# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/my_sample_data')
try:
    os.makedirs(local_download_path)
except: pass
   
for file in file_lists_from_drive:
    print('title: %s, id: %s' % (file['title'], file['id']))
    fname = os.path.join(local_download_path, file['title'])
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': file['id']})
    f_.GetContentFile(fname)
    print(fname)
    file_lists.append(fname)
   
# print file lists
print(file_lists)

title: CM.txt, id: 1QHq6p67jpE56AI5c1HPhJifTpSYaB9Gl
downloading to /root/my_sample_data/CM.txt
/root/my_sample_data/CM.txt
[{'title': 'CM.txt', 'id': '1QHq6p67jpE56AI5c1HPhJifTpSYaB9Gl'}, '/root/my_sample_data/CM.txt']


In [0]:
f = open('../root/my_sample_data/CM.txt', encoding="utf-8")

In [0]:
import re
import kss
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
choseong_list = [char for char in "ㄱㄲㄴㄷㄸㄹㅁㅂㅃㅅㅆㅇㅈㅉㅊㅋㅌㅍㅎ"]

In [17]:
i=0
gogo= 0
sentence_list = []
lowCount = 7
longCount = 20
while True:
    line = f.readline()
    if not line: break
    if "<!DOCTYPE" in line:
        gogo = 0
    if "<text>" in line:
        gogo = 1
    if "</text>" in line:
        gogo = 0
    if gogo == 0: continue
    if line != '\n':
        i=i+1
        line = line.replace(u'\u3000', u"")
        line = line.replace(u'\x80', u"")
        line = line.replace(u'\x84', u"")
        line = line.replace(u'\x88', u"")
        line = line.replace(u'\x8a', u"")
        line = line.replace(u'\x8b', u"")
        line = line.replace(u'\x8c', u"")
        line = line.replace(u'\x90', u"")
        line = line.replace(u'\x94', u"")
        line = line.replace(u'\x97', u"")
        line = line.replace(u'\x9a', u"")
        line = line.replace(u'\xa0', u"")
        line = line.replace(u'\x9e', u"")
        line = line.replace(u'\x9d', u"")
        line = re.sub('<trunc>.+?<\/trunc>', '', line, 0, re.S)
        line = re.sub('<.+?>', '', line, 0, re.S)
        line = re.sub('\(.+?\)', '', line, 0, re.S)
        line = re.sub('\[.+?\]', '', line, 0, re.S)
        line = re.sub('\{.+?\}', '', line, 0, re.S)
        line = re.sub('《.+?》', '', line, 0, re.S)
        line = re.sub('[-§¨©°³¸¼´·²～¯¥=+,#/ꮀ\?:;¡^$@*\"※~&%ㆍ!『』\‘|\(\)\[\]\<\>`\'…《》\{\}_「」±√]', '', line,0,re.S)
        line = line.lower()
        line = re.sub('[a-z]', '', line, 0, re.S)
        line = re.sub('[A-Z]', '', line, 0, re.S)
        line_split = kss.split_sentences(line)
        temp_line = []
        for l_sp in line_split:
            if len(l_sp) >lowCount and len(l_sp) <longCount:
                temp_line.append(l_sp)
        sentence_list+=temp_line
f.close()
print(len(sentence_list))
print(sentence_list[:5])

91444
['뭘 좀 올려야지.', '뭘 좀 올렸어.', '다시 돌려 앞으로.', '우리가족은 아빠 엄마 오빠 나.', '목소리 왜 이케 작어.']


In [18]:
#print(sentence_list[:5])
word_list = []
target = []
for sentence in sentence_list:
    sen2word = ""
    for word in sentence:
        if re.match('[가-힣]',word) is not None :
            sen2word+=choseong_list[(int)((ord(word)-ord('가'))/(21*28))]
        else :
            sen2word+=word
    target.append(sentence+'\n')
    word_list.append(sen2word+'\n')
    
#print(target[:5])
#print(word_list[:5])
src_vocab=set()
for words in word_list: # 1줄씩 읽음
    for char in words: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in target:
    for char in line:
        tar_vocab.add(char)
        
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
#print(src_vocab_size)
#print(tar_vocab_size)
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab)
print(tar_vocab[10:20])
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
#print(src_to_index)
#print(tar_to_index)
encoder_input = []
for line in word_list: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X[:-1])
print(encoder_input[:5])
decoder_input = []
for line in word_list:
    temp_X = []
    for w in line:
      temp_X.append(src_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])
decoder_target = []
for line in target:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_target.append(temp_X)
print(decoder_target[:5])

['\n', ' ', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
['7', '8', '9', '가', '각', '간', '갇', '갈', '감', '갑']
[[20, 2, 26, 2, 25, 19, 25, 26, 3], [20, 2, 26, 2, 25, 19, 25, 3], [17, 23, 2, 17, 19, 2, 25, 25, 19, 3], [25, 19, 14, 26, 25, 2, 25, 22, 2, 25, 20, 2, 25, 22, 2, 16, 3], [20, 23, 19, 2, 25, 2, 25, 29, 2, 26, 25, 3]]
[[20, 2, 26, 2, 25, 19, 25, 26, 3, 1], [20, 2, 26, 2, 25, 19, 25, 3, 1], [17, 23, 2, 17, 19, 2, 25, 25, 19, 3, 1], [25, 19, 14, 26, 25, 2, 25, 22, 2, 25, 20, 2, 25, 22, 2, 16, 3, 1], [20, 23, 19, 2, 25, 2, 25, 29, 2, 26, 25, 3, 1]]
[[636, 2, 1168, 2, 1041, 506, 985, 1201, 3, 1], [636, 2, 1168, 2, 1041, 513, 998, 3, 1], [308, 890, 2, 356, 506, 2, 977, 1095, 517, 3, 1], [1065, 557, 14, 1165, 1097, 2, 964, 743, 2, 1005, 565, 2, 1038, 743, 2, 211, 3, 1], [616, 845, 557, 2, 1055, 2, 1109, 1364, 2, 1125, 998, 3, 1]]


In [19]:
max_src_len = max([len(line) for line in word_list])-1
max_tar_len = max([len(line) for line in target])
print(max_src_len)
print(max_tar_len)
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

encoder_inputer = encoder_input
decoder_inputer = decoder_input
decoder_targeter = decoder_target

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

decoder_inputs = Input(shape=(None, src_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

19
20


In [0]:
encoder_input = to_categorical(encoder_inputer, num_classes=src_vocab_size)
decoder_input = to_categorical(decoder_inputer, num_classes=src_vocab_size)
decoder_target = to_categorical(decoder_targeter, num_classes=tar_vocab_size)

In [21]:
model.fit(x=[encoder_input, decoder_input], y=decoder_target,  epochs=50, validation_split=0.2)

Train on 73155 samples, validate on 18289 samples
Epoch 1/50
73155/73155 [==============================] - 44s 600us/sample - loss: 1.5370 - val_loss: 1.1822
Epoch 2/50
73155/73155 [==============================] - 26s 355us/sample - loss: 1.1407 - val_loss: 1.1327
Epoch 3/50
73155/73155 [==============================] - 26s 354us/sample - loss: 1.0897 - val_loss: 1.1372
Epoch 4/50
73155/73155 [==============================] - 26s 354us/sample - loss: 1.0468 - val_loss: 1.0709
Epoch 5/50
73155/73155 [==============================] - 28s 380us/sample - loss: 1.0128 - val_loss: 1.0437
Epoch 6/50
73155/73155 [==============================] - 35s 484us/sample - loss: 0.9800 - val_loss: 1.0519
Epoch 7/50
73155/73155 [==============================] - 35s 480us/sample - loss: 0.9529 - val_loss: 1.0202
Epoch 8/50
73155/73155 [==============================] - 35s 479us/sample - loss: 0.9309 - val_loss: 1.0036
Epoch 9/50
73155/73155 [==============================] - 35s 480us/sample - l

In [22]:
from os import path
from google.colab import drive

notebooks_dir_name = 'MLTEXT'
drive.mount('/content/gdrive')
notebooks_base_dir = path.join('./gdrive/My Drive/', notebooks_dir_name)
if not path.exists(notebooks_base_dir):
  print('Check your google drive directory. See you file explorer')

model.save(path.join(notebooks_base_dir, "seq2seq.krs"))

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./gdrive/My Drive/MLTEXT/seq2seq.krs/assets


NameError: ignored

In [0]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [0]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [0]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [0]:
def str2cho(input_str):
    #문장을 입력받으면 초성 문자열로반환하는 코드
    cho = ""
    for word in input_str:
        if re.match('[가-힣]',word) is not None :
            cho+=(choseong_list[(int)((ord(word)-ord('가'))/(21*28))])
        else :
            cho+=(word)
    return cho

In [0]:
def decode_sequence(input_seq,strWORD):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)
    # 첫글자에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, src_vocab_size))
    target_seq[0, 0, src_to_index[str2cho(strWORD[0])]] = 1.

    stop_condition = False
    decoded_sentence = ""
    cho_sen = ""
    cho_sen+=str2cho(strWORD[0])
    itr = 0
    while not stop_condition: # stop_condition이 True가 될 때까지 루프 반복
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 입력 글자를 넘으면 중단.
        if (sampled_char == '\n' or
           len(decoded_sentence) > len(strWORD)):
            stop_condition = True
        else:
          itr+=1

        # 길이가 1인 타겟 시퀀스를 업데이트 합니다.
        target_seq = np.zeros((1, 1, src_vocab_size))
        
        target_seq[0, 0, src_to_index[str2cho((strWORD[itr]))]] = 1.
        cho_sen+=str2cho(strWORD[itr])
        # 상태를 업데이트 합니다.
        states_value = [h, c]
    #print(cho_sen)
    return decoded_sentence

In [50]:
i = 0
for seq_index in [105,70000,15002,40015,5]: # 입력 문장의 인덱스
    i+=1
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq,word_list[seq_index])
    print(35 * "-")
    print('입력 문장:', word_list[seq_index])
    print('정답 문장:', target[seq_index][:len(target[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)]) # '\n'을 빼고 출력

-----------------------------------
입력 문장: ㄲ ㄱㅊㅇ ㅈㅅㅇㄴㄷ

정답 문장: 꽤 괜찮은 점수였는데
번역기가 번역한 문장: 꼭 기찮의 자수이니데

-----------------------------------
입력 문장: ㅈㅇㅁㅌㄹㅇㅅ ㅂㅇ ㄱㄱ ㄸㄱㅇㅇ.

정답 문장: 지오메트리에서 배운 것과 똑같아요.
번역기가 번역한 문장: 지오메트리에서 방운 것과 똑같애요.

-----------------------------------
입력 문장: ㄱㄹ ㄱ ㅊ ㅂㅉ ㄱㄹㄱ

정답 문장: 그런 게 첫 번째 그렇고
번역기가 번역한 문장: 그런 거 첫 번째 그렇고

-----------------------------------
입력 문장: ㅎㅌㅅㅇ ㅅㄴ ㅇㄴㄷ

정답 문장: 형태소일 수는 있는데
번역기가 번역한 문장: 화태소에 사는 있는데

-----------------------------------
입력 문장: ㅇㄹㄱ ㄴ ㅁㅇㄷ

정답 문장: 이렇게 네 명인데
번역기가 번역한 문장: 이렇게 난 먹인다



In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None, 33)]   0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None, 33)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 296960      input_3[0][0]                    
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  296960      input_4[0][0]                    
                                                                 lstm_2[0][1]               